# Parameter estimation sensitivity

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from ddn3 import tools, parameter_tuning
import simulation_rev as sim

%load_ext autoreload
%autoreload 2

In [2]:
def make_graph():
    # Structures of the directly graph
    graph_ba_di = sim.barabasi_albert_digraph(41)
    for edge in graph_ba_di.edges:
        u, v = edge
        graph_ba_di.edges[u, v]['weight'] = np.random.choice([-0.5,0.5], 1)

    edges_base = list(graph_ba_di.edges)
    n_edges = len(edges_base)
    n_remove = int(n_edges/4)
    idx_edge_remove = np.random.choice(n_edges, n_remove*2, replace=False)
    idx_remove_lst0 = idx_edge_remove[:n_remove]
    idx_remove_lst1 = idx_edge_remove[n_remove:]

    graph_c0 = graph_ba_di.copy()
    for i in idx_remove_lst0:
        u = edges_base[i][0]
        v = edges_base[i][1]
        graph_c0.remove_edge(u, v)

    graph_c1 = graph_ba_di.copy()
    for i in idx_remove_lst1:
        u = edges_base[i][0]
        v = edges_base[i][1]
        graph_c1.remove_edge(u, v)

    mat_wt0, _, _ = sim.graph_to_matrix(graph_c0)
    mat_wt1, _, _ = sim.graph_to_matrix(graph_c1)

    return mat_wt0, mat_wt1


load_saved = True
if load_saved:
    tmp = np.load("./temp/struct_20240325.npz")
    mat_wt0 = tmp["mat_wt0"]
    mat_wt1 = tmp["mat_wt1"]
else:
    mat_wt0, mat_wt1 = make_graph()
    np.savez("./temp/struct.npz", mat_wt0=mat_wt0, mat_wt1=mat_wt1)


In [3]:
tuning_methods = ["cv_joint", "cv_sequential", "cv_bai", "mb_cv", "mb_bai"]

Baseline

In [14]:
n_rep = 10
res_base = np.zeros((n_rep, len(tuning_methods), 2))

for n in range(n_rep):
    print(n)
    n1 = 100
    n2 = 100
    dat0 = sim.sim_steady_state_linear_gaussian(mat_wt0, n1)
    dat1 = sim.sim_steady_state_linear_gaussian(mat_wt1, n2)
    dp = parameter_tuning.DDNParameterSearch(
        dat0,
        dat1,
        n_cv=10,
        lambda1_list=np.arange(0.025, 0.725, 0.025),
        lambda2_list=np.arange(0.025, 0.225, 0.025),
        ratio_validation=0.5,
        err_scale=0,
    )
    for idx, m in enumerate(tuning_methods):
        _, rho1, rho2 = dp.fit(m)
        res_base[n, idx] = [rho1, rho2]

np.savez("./temp/baseline_20240325.npz", res=res_base, method=tuning_methods)


In [16]:
# np.mean(res_base, axis=0)

Noise in the weights

In [27]:
n_rep = 10
res_base = np.zeros((n_rep, len(tuning_methods), 2))

for n in range(n_rep):
    print(n)
    n1 = 100
    n2 = 100

    idx0 = np.where(mat_wt0!=0)
    noise = np.random.uniform(-0.1, 0.1, len(idx0[0]))
    mat_wt0_noise = np.copy(mat_wt0)
    mat_wt0_noise[idx0] += noise

    idx1 = np.where(mat_wt1!=0)
    noise = np.random.uniform(-0.1, 0.1, len(idx1[0]))
    mat_wt1_noise = np.copy(mat_wt1)
    mat_wt1_noise[idx1] += noise 

    dat0 = sim.sim_steady_state_linear_gaussian(mat_wt0_noise, n1)
    dat1 = sim.sim_steady_state_linear_gaussian(mat_wt1_noise, n2)
    dp = parameter_tuning.DDNParameterSearch(
        dat0,
        dat1,
        n_cv=10,
        lambda1_list=np.arange(0.025, 0.725, 0.025),
        lambda2_list=np.arange(0.025, 0.225, 0.025),
        ratio_validation=0.5,
        err_scale=0,
    )
    for idx, m in enumerate(tuning_methods):
        _, rho1, rho2 = dp.fit(m)
        res_base[n, idx] = [rho1, rho2]

np.savez("./temp/weights_20240325.npz", res=res_base, method=tuning_methods)


Sample number variation

In [28]:
n_rep = 10
res_base = np.zeros((n_rep, len(tuning_methods), 2))

for n in range(n_rep):
    print(n)
    n1 = int(np.random.uniform(80, 120))
    n2 = int(np.random.uniform(80, 120))
    dat0 = sim.sim_steady_state_linear_gaussian(mat_wt0, n1)
    dat1 = sim.sim_steady_state_linear_gaussian(mat_wt1, n2)
    dp = parameter_tuning.DDNParameterSearch(
        dat0,
        dat1,
        n_cv=10,
        lambda1_list=np.arange(0.025, 0.725, 0.025),
        lambda2_list=np.arange(0.025, 0.225, 0.025),
        ratio_validation=0.5,
        err_scale=0,
    )
    for idx, m in enumerate(tuning_methods):
        _, rho1, rho2 = dp.fit(m)
        res_base[n, idx] = [rho1, rho2]

np.savez("./temp/samplesize_20240325.npz", res=res_base, method=tuning_methods)


Non-Gaussian noise

In [79]:
# xx = np.random.beta(0.5,0.5,1000) + np.random.randn(1000)*0.1
# _ = plt.hist(xx)
# print(np.std(xx))

In [80]:
# xx = np.random.gamma(2,2,10000)
# _ = plt.hist(xx, 100)

In [12]:
# dat0 = sim.sim_steady_state_linear_nongaussian(mat_wt0, 10000)
# plt.hist(dat0[:,1])

In [13]:
n_rep = 10
res_base = np.zeros((n_rep, len(tuning_methods), 2))

for n in range(n_rep):
    print(n)
    n1 = 100
    n2 = 100
    dat0 = sim.sim_steady_state_linear_nongaussian(mat_wt0, n1)
    dat1 = sim.sim_steady_state_linear_nongaussian(mat_wt1, n2)
    dp = parameter_tuning.DDNParameterSearch(
        dat0,
        dat1,
        n_cv=10,
        lambda1_list=np.arange(0.025, 0.725, 0.025),
        lambda2_list=np.arange(0.025, 0.225, 0.025),
        ratio_validation=0.5,
        err_scale=0,
    )
    for idx, m in enumerate(tuning_methods):
        _, rho1, rho2 = dp.fit(m)
        res_base[n, idx] = [rho1, rho2]

np.savez("./temp/est_nongaussian_20240325.npz", res=res_base, method=tuning_methods)


## Analysis

In [6]:
res_base = np.load("./temp/est_baseline_20240325.npz")['res']
res_weight = np.load("./temp/est_weights_20240325.npz")['res']
res_sample = np.load("./temp/est_samplesize_20240325.npz")['res']
res_nongaussian = np.load("./temp/est_nongaussian_20240325.npz")['res']
tuning_methods = ["cv_joint", "cv_sequential", "cv_bai", "mb_cv", "mb_bai"]

In [11]:
met_idx = 0

for met_idx in range(5):

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

    labels = ['Baseline', 'Weight', 'Sample', 'Distribution']

    for rho_idx in range(2):
        x0 = res_base[:, met_idx, rho_idx]
        x1 = res_weight[:, met_idx, rho_idx]
        x2 = res_sample[:, met_idx, rho_idx]
        x3 = res_nongaussian[:, met_idx, rho_idx]

        all_data = [x0, x1, x2, x3]
        axs[rho_idx].violinplot(all_data, showmeans=True)    
        if rho_idx==0:
            axs[rho_idx].set_ylim([0,1])
        else:
            axs[rho_idx].set_ylim([0,0.5])

    for ax in axs:
        ax.yaxis.grid(True)
        ax.set_xticks([y + 1 for y in range(len(all_data))], labels=labels)
    
    axs[0].set_title('$\lambda_1$')
    axs[1].set_title('$\lambda_2$')

    fig.savefig(f"./temp/sensi_{tuning_methods[met_idx]}.png")


## Example

In [11]:
dat0 = sim.sim_steady_state_linear_gaussian(mat_wt0, n1)
dat1 = sim.sim_steady_state_linear_gaussian(mat_wt1, n2)

dp = parameter_tuning.DDNParameterSearch(
    dat0,
    dat1,
    n_cv=5,
    lambda1_list=np.arange(0.025, 0.725, 0.025),
    lambda2_list=np.arange(0.025, 0.225, 0.025),
    ratio_validation=0.5,
    err_scale=0.0,
)

In [ ]:
val_err2d, rho1, rho2 = dp.fit("cv_joint")

In [ ]:
plt.imshow(np.mean(val_err2d, axis=0))

In [ ]:
print(rho1, rho2)

In [12]:
[val_err1, val_err2], rho1, rho2 = dp.fit("cv_sequential")

In [13]:
plt.plot(val_err1.T)

In [14]:
plt.plot(np.mean(val_err2, axis=0))

## Ground truth
For debug

In [43]:
comm_gt, diff_gt = tools.get_common_diff_net_topo([mat_wt0, mat_wt1])

In [59]:
n1 = 100
n2 = 100

dat0 = sim.sim_steady_state_linear_gaussian(mat_wt0, n1)
dat1 = sim.sim_steady_state_linear_gaussian(mat_wt1, n2)

In [60]:
from ddn3 import ddn, performance

# omega1, omega2 = ddn.ddn(dat0, dat1, lambda1=0.3, lambda2=0.1)

lambda1_rg = np.arange(0.02, 1.01, 0.02)
res_comm_ddn = np.zeros((len(lambda1_rg), 5))
res_diff_ddn = np.zeros((len(lambda1_rg), 5))
for i, lamb in enumerate(lambda1_rg):
    out_ddn = ddn.ddn(dat0, dat1, lambda1=lamb, lambda2=0.065)
    comm_est, diff_est = tools.get_common_diff_net_topo(out_ddn)
    res_comm_ddn[i] = performance.get_error_measure_two_theta(comm_est, comm_gt)
    res_diff_ddn[i] = performance.get_error_measure_two_theta(diff_est, diff_gt)


In [61]:
plt.plot(res_comm_ddn[:,0])
plt.plot(res_comm_ddn[:,1])
plt.ylim([0,50])

In [62]:
plt.plot(res_diff_ddn[:,0])
plt.plot(res_diff_ddn[:,1])
plt.ylim([0,50])

## Misc
Different ways of generating Gaussian samples
- Conditional independence

In [ ]:
prec_mat = np.eye(21)
for i in range(20):
    prec_mat[i, i+1] = 0.4
    prec_mat[i+1, i] = 0.4

cov_mat = np.linalg.inv(prec_mat)


In [ ]:
plt.imshow(cov_mat)
plt.colorbar()

In [ ]:
cov_mat_chol = np.linalg.cholesky(cov_mat)

In [ ]:
plt.imshow(cov_mat_chol)